In [1]:
# %load Simulation_brute.py
import numpy as np
import pandas as pd

In [2]:
from make_demo import (generate_SexeAge, generate_Handicap)
from make_travail import (generate_Activite, 
                          generate_Emploi, 
                          add_Salaire,
                          add_Retraite,
                          generate_Etudiants)
from make_couple import (generate_Couple,
                        generate_pop_men)                          

sample_size_target = 1000

### Génère la population : âge et sexe

Input : pop_age_sexe_2016.csv     
Table générée à partir de pop-1janvier-fe.xls (https://www.insee.fr/fr/statistiques/1892086)
Source : Insee, estimations de population (résultats provisoires arrêtés à fin 2015).    
Champ : France y compris Mayotte.


In [3]:
# AgeSexe   
effectifs_age_sexe = pd.read_csv("data/demographie/pop_age_sexe_2016.csv")
generation = generate_SexeAge(effectifs_age_sexe, sample_size_target)
effectifs_age_sexe = generation[1]
population = generation[0]

sample_size = len(population)
max_age = effectifs_age_sexe['age'].max()

Test effectifs simulés pour âge et sexe :
count    188.000000
mean       1.003014
std        0.023816
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.200000
Name: ratio, dtype: float64


In [5]:
effectifs_age_sexe.head()

,age,sexe,effectif_ref
0,0,femme,374179
1,1,femme,385442
2,2,femme,386831
3,3,femme,391853
4,4,femme,399632


In [6]:
### Activité
reference_activite = pd.read_csv("data/demographie/activite_2015.csv")   
population['activite']= generate_Activite(reference_activite, effectifs_age_sexe, population, sample_size)

#Emploi : à partir du taux de chomaĝe
reference_emploi = pd.read_csv("data/travail/chomage.csv")


population['emploi'] = generate_Emploi(reference_emploi, population, max_age)

# Salaire
reference_salaire = pd.read_csv("data/travail/salaire_brut_horaire.csv")
population['salaire'] = add_Salaire(reference_salaire,population, max_age)

# Retraite
reference_retraite = pd.read_csv("data/travail/retraite_2012.csv")
population['retraite'] = add_Retraite(reference_retraite, population, max_age)


#Étudiants
reference_etudes = pd.read_csv("data/demographie/etudes.csv")
population['etudes'] = generate_Etudiants(reference_etudes, population)
# Selon une étude INSEE (2016) -> 23%/
# http://www.insee.fr/fr/themes/document.asp?reg_id=0&ref_id=ip1603#inter6

#### Handicap

reference_handicap = pd.read_csv("data/demographie/handicap_pop.csv")
reference_handicap_jeune = pd.read_csv("data/demographie/handicap_pop_jeune.csv")
population['handicap'] = generate_Handicap(reference_handicap, reference_handicap_jeune, population, effectifs_age_sexe, sample_size)

#### Statut marital
reference_marital = dict()
for sexe in ['homme', 'femme']:
    reference_marital[sexe] = pd.read_csv("data/menages/statut_marital_{0}.csv".format(sexe))
population['statut_marital'] = generate_Couple(reference_marital, population)


''''
TABLE MÉNAGE
'''''

population_menage = generate_pop_men(population)

Test effectifs simulés pour activité :
count    18.000000
mean      0.934389
std       0.214709
min       0.333333
25%       0.899687
50%       1.000000
75%       1.056494
max       1.176471
Name: ratio, dtype: float64
Test effectifs simulés pour emploi :
count    6.000000
mean     0.981972
std      0.234852
min      0.752840
25%      0.793336
50%      0.924561
75%      1.172471
max      1.285170
Name: ratio, dtype: float64
Taux de chômage généré :  10.6728538283
 Pourcentage d'étudiants en emploi 13.1578947368
Test effectifs simulés pour activité :
count    23.000000
mean      0.834942
std       0.288731
min       0.333333
25%       0.777778
50%       0.819820
75%       0.829870
max       2.000000
Name: ratio, dtype: float64
 Pourcentage d'handicapés en emploi 63.63636363636363
 Pourcentage d'handicapés en activite 72.72727272727273
Proportion générée des statuts maritaux :
0    0.410539
1    0.438725
2    0.075980
3    0.074755
Name: statut_marital, dtype: float64
